In [20]:
#import pandas and numpy libraries
import pandas as pd
import numpy as np
import sys #sys needed only for python version
#import gaussian naive bayes from scikit-learn
import sklearn as sk
#seaborn for pretty plots
import seaborn as sns
import os

from sklearn import metrics


In [34]:


def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )



In [15]:
# nacitanie dat 
df_slide1 = pd.read_csv('size=25/sliding1.csv', sep=',', header=None,
                 error_bad_lines=False, warn_bad_lines=True, low_memory=False)



In [41]:
directory = 'size=25/'

frames = []
allcsvFiles = []
for root,dirs, files in os.walk(directory):
    for file in log_progress(files,every=1):
       allcsvFiles = files
       #if (file.endswith(".csv")):
       #    f=open(directory + file, 'r')
       #    # perform calculation
       #    df1 = pd.read_csv(f,sep=',')
       #    frames.append(df1)
       # 
       #    f.close()

In [47]:
print(allcsvFiles)

['sliding1.csv', 'sliding10.csv', 'sliding11.csv', 'sliding12.csv', 'sliding13.csv', 'sliding14.csv', 'sliding15.csv', 'sliding16.csv', 'sliding17.csv', 'sliding18.csv', 'sliding19.csv', 'sliding2.csv', 'sliding20.csv', 'sliding3.csv', 'sliding4.csv', 'sliding5.csv', 'sliding6.csv', 'sliding7.csv', 'sliding8.csv', 'sliding9.csv']


In [51]:
def getint(name):
    basename = name.partition('.')[0]
    alpha, num = basename.split('g')
    return int(num)
allcsvFiles = sorted(allcsvFiles, key=getint)


In [52]:
directory = 'size=25/'
frames = []
for file in log_progress(allcsvFiles,every=1):
       if (file.endswith(".csv")):
           f=open(directory + file, 'r')
           df1 = pd.read_csv(f,sep=',')
           frames.append(df1)
           f.close()

In [53]:
# zoradenie frames
resultcsv = pd.concat(frames)


In [75]:
# SAVE TO CSV FILE
#print(resultcsv)
#resultcsv = resultcsv.reset_index()
#del resultcsv['Unnamed: 0']

##resultcsv.to_csv(directory + 'result.csv', sep=',')
#print(resultcsv.columns)
cols = [col for col in resultcsv.columns if col not in ['movement','user']]

print(cols)

['0_x', '0_y', '0_z', '1_x', '1_y', '1_z', '2_x', '2_y', '2_z', '3_x', '3_y', '3_z', '4_x', '4_y', '4_z', '5_x', '5_y', '5_z', '6_x', '6_y', '6_z', '7_x', '7_y', '7_z', '8_x', '8_y', '8_z', '9_x', '9_y', '9_z', '10_x', '10_y', '10_z', '11_x', '11_y', '11_z', '12_x', '12_y', '12_z', '13_x', '13_y', '13_z', '14_x', '14_y', '14_z', '15_x', '15_y', '15_z', '16_x', '16_y', '16_z', '17_x', '17_y', '17_z', '18_x', '18_y', '18_z', '19_x', '19_y', '19_z', '20_x', '20_y', '20_z', '21_x', '21_y', '21_z', '22_x', '22_y', '22_z', '23_x', '23_y', '23_z', '24_x', '24_y', '24_z']


In [76]:

from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(resultcsv[cols], resultcsv['movement'], test_size=0.2)

#initialize and fit the naive bayes classifier
from sklearn.naive_bayes import GaussianNB
skgnb = GaussianNB()

In [82]:
#print(y_train)

skgnb.fit(x_train,y_train)
train_predict = skgnb.predict(x_train)
#print(train_predict))

In [83]:
print(train_predict)

[4 2 2 ..., 7 0 0]


In [86]:
#see how accurate the training data was fit
print("Training accuracy:",metrics.accuracy_score(y_train, train_predict))

#use the trained model to predict the test values
test_predict = skgnb.predict(x_test)
print("Testing accuracy:",metrics.accuracy_score(y_test, test_predict))

Training accuracy: 0.267177323956
Testing accuracy: 0.26783314051


**naive-bayes na vzorke test_size=33** 
* aj metriky


In [87]:
x_train, x_test, y_train, y_test = train_test_split(resultcsv[cols], resultcsv['movement'], test_size=0.33)
skgnb = GaussianNB()
skgnb.fit(x_train,y_train)
train_predict = skgnb.predict(x_train)
test_predict = skgnb.predict(x_test)

print("Training accuracy:",metrics.accuracy_score(y_train, train_predict))
print("Testing accuracy:",metrics.accuracy_score(y_test, test_predict))

Training accuracy: 0.267741382214
Testing accuracy: 0.266686415147


In [95]:
print("\nClassification Report:")
#print(metrics.classification_report(y_test, test_predict, target_names=['movement','user']))
print("\nConfusion Matrix:")
skcm = metrics.confusion_matrix(y_test,test_predict)
#putting it into a dataframe so it prints the labels
# skcm = pd.DataFrame(skcm, columns=['movement'])
# skcm['actual'] = ['edible','poisonous']
# skcm = skcm.set_index('actual')


Classification Report:

Confusion Matrix:


ValueError: Shape of passed values is (8, 8), indices imply (1, 8)